In [168]:
import csv
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from lxml import html
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import dateutil
from dateutil.parser import *
from datetime import datetime
import pathlib
from htmldate import find_date
import trafilatura

In [49]:
# process statewise UK VERSION --> type "General"


def process_state_UK(state):

    agile_regex = re.compile(r'\bagility\b|\bagile\b', re.IGNORECASE | re.UNICODE)

    agile_method_regex = re.compile(r'\bagile\b(?:(?![.]|agile).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)

    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE)

    digital_regex = re.compile(r'\bdigital\b|\bditialization\b|\bdigitalisation\b\bdigital transformation\b', re.IGNORECASE | re.UNICODE)

    military_regex = re.compile(r'\bmilitary\b|\barmy\b|\barmed forces\b|\bnavy\b|\binfantery\b', re.IGNORECASE | re.UNICODE)

    #searchsite_regex = re.compile(r'search', re.IGNORECASE | re.UNICODE)

    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/UK/SAVEPLACE/agile_sites_output_UK_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])
    array = {}
    for line in csv_main: 
        array[line["id"]] = line
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/TextFiles/UK/{}".format(state)
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)

    for id, line in array.items():
        print("processing {}".format(id))
        id = line["id"]
        domain = line["domain"]
        url = line["url"]
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/UK/SAVEPLACE/{}/{}/{}.html'.format(state, domain, id)
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            soup = BeautifulSoup(open(path, encoding='windows-1252'), "html.parser")    
        

        #Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        

        # Getting date6

        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()



        #deleting all non-body text content from the html

        for script in soup(['script', 'style', 'a', 'meta', 'head', 'footer', 'navbar', 'header', 'search-box', 'global-cookie-message', 'id="global-cookie-message"', 'global-bar', 'menu', 'noscript', 'global-cookie-message', 'search']):
            script.decompose()    # rip it out
            
        # get text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
        
        #old 'simple' cleaning 
        #doctext = soup.get_text().replace("\n", " ")

        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        

        #Getting date5

        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
        
        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier

        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")

  
        #different parsing for for htmldate and trafilatura

        try:
            html_tree = html.parse(open(path))
        except UnicodeDecodeError:
            html_tree = html.parse(open(path, encoding='windows-1252'))
            

        #Getting date with the htmldate package
        htmldate = find_date(html_tree)

        #since html date gives out nonetype when it doesn't find anything, it has to be respecified to empty string so that it works with dateparser later on
        if htmldate is None:
            htmldate = ''

        #htmldate = htmldate.astype('str')

        #Getting the main content of the page with the trafilatura package
        #main_content = trafilatura.extract(html_tree)


        # assinging the already crawled dates
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7, htmldate]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 


        #finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext.lower())
        

        #finding the matches for agile...methods
        agile_method = []
        agile_method = agile_method_regex.findall(doctext.lower())

         #finding the matches for digital
        digital_term = []
        digital_term = digital_regex.findall(doctext.lower())

        #finding the matches for military
        military_term = []
        military_term  = military_regex.findall(doctext.lower())



        # finding the context for the agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all the variables to the items

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)
        
        if len(agile_method) == 0:
            line["agile_method"] = "" 
        else:
            line["agile_method"] = ",".join(agile_method)

        if len(digital_term) == 0:
            line["digital_term"] = "" 
        else:
            line["digital_term"] = ",".join(digital_term)

        if len(military_term) == 0:
            line["military_term"] = "" 
        else:
            line["military_term"] = ",".join(military_term)


        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "UK"
        line["level"] = state
        #line["main_content"] = main_content
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
        line['doctext'] = doctext
        #line['is_search'] = is_search
        
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/{}.csv".format(state), "w")
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'htmldate', 'final_date', 'heading', 'agile_term', 'agile_method', 'agile_context_pre', 'agile_context_post', 'digital_term', 'military_term', 'text_file_loc', 'doctext'])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()
    

In [50]:
#Running processing for UK
process_state_UK("General")

273-11f5-389a-9fdc-bb25698ad02d
processing 129f6cc5-c486-3e87-ba17-f7a9e459cbe7
processing 6623951c-ab58-3e64-946a-711208cb09ab
processing 311bd1b2-fbb8-361c-a0bd-10df02957190
processing 994bb0f2-1762-346a-8458-b2086eaeb744
processing c8e6259e-d532-3be8-b915-64a31a58219d
processing 301dcd04-e8a8-3478-951d-dd1fbcfdbd32
processing 3bd98b3e-6e08-38fd-84a2-312b4e70a36d
processing dae049dc-8dcf-37ca-9746-c5695c348994
processing 10296bf6-9750-30b7-87f9-d616afaff68d
processing 00fbf2a7-1af1-3636-88c6-7d9df0085a8e
processing c1c89022-02b3-3093-8eeb-c1c2408bbbd7
processing d33657af-96f3-3db2-9247-d38a8dc31f2a
processing f953464b-6d01-3672-a78a-6bae348abb79
processing 474df3c9-7253-3377-93d4-d9bdc4972bec
processing fd3373c3-4fd7-32b0-bd98-d8263b7c70f2
processing 6434a58a-6778-33db-9992-7784b5cb9fc4
processing 06321551-224b-30b3-8384-c93cbc7bb417
processing 3546db0c-a883-3a7b-bae3-1beedb3b93e7
processing 15c44474-626e-31e9-ade6-691a38508173
processing 627cc0eb-abbf-3377-aacd-e417f79e3d08
processi

In [97]:
##### Cleaning State UK
def cleaning_state_UK(state):
    inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/{}.csv".format(state)
    outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/{}_v1.csv".format(state)

    with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
        df = pd.read_csv(inFile)
        df.sort_values(by=['final_date'])
        df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term']), inplace=True)
        df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df['agile_context_pre'] = df['agile_context_pre'].astype(str)
        df['agile_context_post'] = df['agile_context_post'].astype(str)
        df['heading'] = df['heading'].astype(str)
        df['domain'] = df['domain'].astype(str)
        #deleting search terms
        df = df[~df.url.str.contains('/search/')]
        #deleting false positives
        df = df[~df.agile_context_post.str.startswith("(")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith(")")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith("EB")] # supplier list
        df = df[~df.agile_context_post.str.match('and battle-winning armed forces')] #army related agility
        df = df[~df.agile_context_pre.str.match('Atom CategoriesCategories Select Category')] # dopdown menu entry
        df = df[~df.agile_context_pre.str.match('RAF to Force Generate')] # Entry on Air Commanders
        df = df[~df.agile_context_post.str.startswith("Trains")] # name of a company "Agility Trains"
        df = df[~df.agile_context_post.str.startswith("trains")] # name of a company "Agility Trains"
        df = df[~df.agile_context_post.str.match('People Services Limited Independent')] #company name
        df = df[~df.heading.str.match('Fitness classes')]
        #deleting csv previews
        df = df[~df.domain.str.endswith("csv/preview")]
        #deleting things that have to do with military 
        df = df[df.military_term.isnull()]
        #deleting cases where no date has been found (htmldate attributes 2020-01-01 for no dates)
        df = df[df['final_date'] != '2020-01-01 00:00:00']
        #deleting all cases from 2020 because the year has just started
        df = df[df['final_date'] < '2020-01-01 00:00:00']
        df['final_date'] = pd.to_datetime(df['final_date'])
        df.to_csv(outFile)
        return df

In [98]:
# Cleaning State UK
cleaning_state_UK('General')

,id,country,level,url,domain,date1,date2,date3,date4,date5,...,final_date,heading,agile_term,agile_method,agile_context_pre,agile_context_post,digital_term,military_term,text_file_loc,doctext
3,48f7ab58-cabb-3461-9dcd-0ad29d6fae28,UK,General,https://www.gov.uk/guidance/accessibility-requ...,www.gov.uk,2019-05-17T15:40:09.000+01:00,NaN,NaN,NaN,2018-05-09T09:28:00.000,...,2018-05-09 00:00:00,Understanding new accessibility requirements f...,agile,NaN,If you’re using an,approach to redeveloping an,digital,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...
6,ecb03a68-22dc-36a4-b0c9-e5e1a123a0bd,UK,General,https://www.gov.uk/guidance/guidance-for-opera...,www.gov.uk,2016-02-15T11:32:40.000+00:00,NaN,NaN,NaN,2013-01-04T14:54:00.000,...,2013-01-04 00:00:00,New nuclear power: support for industry develo...,agility,NaN,"of government, has increased",to address the regulatory,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...
10,6ddf8e67-c2c5-3901-b212-6aeb67417d5f,UK,General,https://www.gov.uk/government/publications/civ...,www.gov.uk,NaN,NaN,NaN,NaN,2018-08-08T23:00:13.000,...,2018-08-08 23:00:13,3. THE SOCIAL SECTOR: supporting charities and...,agile,NaN,"a responsive, resilient, and",social sector. The government,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...
12,10be72cd-dcf9-376d-8f86-9b9d7a60e0f4,UK,General,https://www.gov.uk/government/speeches/eureka-...,www.gov.uk,NaN,NaN,NaN,NaN,2019-05-14T10:58:00.000,...,2019-05-14 00:00:00,EUREKA Global Innovation Summit 2019,"agile,agile,agile,agile",NaN,and to provide an,platform that can adapt,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...
13,9494ba7d-bc3e-328f-b123-87574faaaa94,UK,General,https://www.gov.uk/government/speeches/the-uk-...,www.gov.uk,NaN,NaN,NaN,NaN,2019-10-28T12:13:17.000,...,2019-10-28 00:00:00,The UK Financial Services Beyond Brexit Summit...,agile,NaN,why we need an,and responsive regulatory approach.,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,c2684060-e94d-3f3a-8437-cd013eaa030b,UK,General,https://gds.blog.gov.uk/2014/10/15/building-di...,gds.blog.gov.uk,2014-10-15T12:00:00+01:00,NaN,NaN,NaN,NaN,...,2014-10-15 00:00:00,BlogGovernment Digital Service,"agile,agile,agile,agile,agile",NaN,about working in an,way. You can adjust,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...
2376,ebefeb60-844f-3c4f-bc35-9380fb5286fb,UK,General,https://digitalpeople.blog.gov.uk/2017/03/08/d...,digitalpeople.blog.gov.uk,2017-03-08T16:26:53+00:00,NaN,NaN,NaN,NaN,...,2017-03-08 00:00:00,BlogDigital people,agile,NaN,is very un-corporate and,which I like. Registers,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...
2379,b1bacb13-a7e0-34f8-9bf9-2e4202c511cc,UK,General,https://gds.blog.gov.uk/2013/07/18/transformin...,gds.blog.gov.uk,2013-07-18T09:30:47+01:00,NaN,NaN,NaN,NaN,...,2013-07-18 00:00:00,BlogGovernment Digital Service,agile,NaN,"ways of working, developing",skills and transforming how,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...
2380,4b00f33c-876b-3d78-9dcc-97b98a3e1c2d,UK,General,https://gds.blog.gov.uk/2012/12/13/meet-the-tr...,gds.blog.gov.uk,2012-12-13T11:37:13+00:00,NaN,NaN,NaN,NaN,...,2012-12-13 00:00:00,BlogGovernment Digital Service,"agile,agile,agile,agile",agile methodology,also favour smaller organisations.,in action - Student

In [113]:
##### Adding Organisation Names based on domain
inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_v1.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_v2.csv"
domains_dict = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/List_of_gov.uk_domain_names_as_at_28_Oct_2019.csv")

domain_regex = re.compile(r'http(s?)://(?!.*http)(www\.)?([a-zA-Z0-9\-.]+)', re.IGNORECASE | re.UNICODE)

def extract(url):
    match = domain_regex.search(url)
    if match:
        match_domain = match.group(3)
        return '.'.join(match_domain.split('.')[-3:])
    else:
        print(url)
        return ''

with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    df = pd.read_csv(inFile)
    df['domain_mod'] = df.apply(lambda row: extract(row['url']), axis=1)
    df2 = pd.merge(df, domains_dict, left_on = 'domain_mod', right_on='Domain: Domain Name', how='left')

    df2.to_csv(outFile)
df2

,Unnamed: 0,id,country,level,url,domain,date1,date2,date3,date4,...,agile_context_pre,agile_context_post,digital_term,military_term,text_file_loc,doctext,domain_mod,Domain: Domain Name,Organisation,Registered for
0,3,48f7ab58-cabb-3461-9dcd-0ad29d6fae28,UK,General,https://www.gov.uk/guidance/accessibility-requ...,www.gov.uk,2019-05-17T15:40:09.000+01:00,NaN,NaN,NaN,...,If you’re using an,approach to redeveloping an,digital,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...,gov.uk,gov.uk,General Domain,General Domain
1,6,ecb03a68-22dc-36a4-b0c9-e5e1a123a0bd,UK,General,https://www.gov.uk/guidance/guidance-for-opera...,www.gov.uk,2016-02-15T11:32:40.000+00:00,NaN,NaN,NaN,...,"of government, has increased",to address the regulatory,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...,gov.uk,gov.uk,General Domain,General Domain
2,10,6ddf8e67-c2c5-3901-b212-6aeb67417d5f,UK,General,https://www.gov.uk/government/publications/civ...,www.gov.uk,NaN,NaN,NaN,NaN,...,"a responsive, resilient, and",social sector. The government,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...,gov.uk,gov.uk,General Domain,General Domain
3,12,10be72cd-dcf9-376d-8f86-9b9d7a60e0f4,UK,General,https://www.gov.uk/government/speeches/eureka-...,www.gov.uk,NaN,NaN,NaN,NaN,...,and to provide an,platform that can adapt,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...,gov.uk,gov.uk,General Domain,General Domain
4,13,9494ba7d-bc3e-328f-b123-87574faaaa94,UK,General,https://www.gov.uk/government/speeches/the-uk-...,www.gov.uk,NaN,NaN,NaN,NaN,...,why we need an,and responsive regulatory approach.,NaN,NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,Tell us whether you accept cookies\nWe use\nab...,gov.uk,gov.uk,General Domain,General Domain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,2374,c2684060-e94d-3f3a-8437-cd013eaa030b,UK,General,https://gds.blog.gov.uk/2014/10/15/building-di...,gds.blog.gov.uk,2014-10-15T12:00:00+01:00,NaN,NaN,NaN,...,about working in an,way. You can adjust,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...,blog.gov.uk,blog.gov.uk,Cabinet Office,gov.uk blogging platform
620,2376,ebefeb60-844f-3c4f-bc35-9380fb5286fb,UK,General,https://digitalpeople.blog.gov.uk/2017/03/08/d...,digitalpeople.blog.gov.uk,2017-03-08T16:26:53+00:00,NaN,NaN,NaN,...,is very un-corporate and,which I like. Registers,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...,blog.gov.uk,blog.gov.uk,Cabinet Office,gov.uk blogging platform
621,2379,b1bacb13-a7e0-34f8-9bf9-2e4202c511cc,UK,General,https://gds.blog.gov.uk/2013/07/18/transformin...,gds.blog.gov.uk,2013-07-18T09:30:47+01:00,NaN,NaN,NaN,...,"ways of working, developing",skills and transforming how,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...,blog.gov.uk,blog.gov.uk,Cabinet Office,gov.uk blogging platform
622,2380,4b00f33c-876b-3d78-9dcc-97b98a3e1c2d,UK,General,https://gds.blog.gov.uk/2012/12/13/meet-the-tr...,gds.blog.gov.uk,2012-12-13T11:37:13+00:00,NaN,NaN,NaN,...,also favour smaller organisations.,in action - Student,"digital,digital,digital,digital,digital,digita...",NaN,/Users/mxm/Google Drive/Masterstudium/Inhalte/...,GOV.UK blogs use cookies to make the site simp...,blog.gov.uk,blog.gov.uk,Cabinet Office,gov.uk blogging platform


In [110]:
domains_dict

,Domain: Domain Name,Organisation,Registered for
0,101.gov.uk,Home Office,The Single Non-Emergency Number (SNEN)
1,aaib.gov.uk,Air Accidents Investigation Branch,Air Accidents Investigation Branch
2,abbotslangley-pc.gov.uk,Abbots Langley Parish Council,Abbots Langley Parish Council
3,abbottsann-pc.gov.uk,Abbotts Ann Parish Council,Abbotts Ann Parish Council
4,aberdeencity.gov.uk,The Aberdeen City Council,Aberdeen City Council
...,...,...,...
3121,yorkconsort.gov.uk,FOSS Internal Drainage Board,York Consortium of Drainage Boards
3122,yorkshirelca.gov.uk,Yorkshire Local Councils Associations,Yorkshire Local Councils Associations
3123,yourpension.gov.uk,Department for Work and Pensions,Department for Work and Pensions
3124,youthjusticeagencyni.gov.uk,Northern Ireland Office,Youth Justice Agency of Northern Ireland


In [82]:
domains_dict['Domain: Domain Name'] = domains_dict['Domain: Domain Name'].astype('str')

domains_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3126 entries, 0 to 3125
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Domain: Domain Name  3126 non-null   object
 1   Organisation         3126 non-null   object
 2   Registered for       3125 non-null   object
dtypes: object(3)
memory usage: 73.4+ KB


In [ ]:
# Testing for Visualisation

21
[[<matplotlib.axes._subplots.AxesSubplot object at 0x7fc659ae8a90>
General Domain
General Domain
Cabinet Office
Cabinet Office
Ofgem
Ofgem
The Executive Office
The Executive Office
Ministry of Justice
Ministry of Justice
HM Land Registry
HM Land Registry
The National Archives
The National Archives
Department for Business, Energy & Industrial Strategy
Department for Business, Energy & Industrial Strategy
The Water Services Regulation Authority
The Water Services Regulation Authority
Human Tissue Authority
Human Tissue Authority
Crown Commercial Service
Crown Commercial Service
Home Office
Home Office
Department for Transport
Department for Transport
Leicester City Council
Leicester City Council
City and County of Swansea Council
City and County of Swansea Council
Stoke-on-Trent City Council
Stoke-on-Trent City Council
Vale of Glamorgan Council
Vale of Glamorgan Council
Greater London Authority
Greater London Authority
Department for Environment, Food & Rural Affairs
Department for En

In [167]:
# printing 
import numpy as np
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib
import seaborn as sns
import math

font = {'weight' : 'bold',
        'size'   : 22}

#matplotlib.rc('font', **font)
#matplotlib.rc('xtick', labelsize=20) 
#matplotlib.rc('ytick', labelsize=20) 


inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_v2.csv"
df=pd.read_csv(inputFileName, parse_dates=["final_date"])
numgroups = len(df.dropna(subset=['Organisation'])[["Organisation"]].groupby('Organisation').size())
print(numgroups)
fig, axs = plt.subplots(math.ceil(numgroups/2), 2)
fig.tight_layout()
fig.set_size_inches(20,120)
print(axs.flat)
print(axs)
orgs = df.dropna(subset=['Organisation'])['Organisation'].unique()

for (org, ax) in zip(orgs, axs.flat):
        print(org)
        print(org)
        df[df.Organisation == org]['final_date'].groupby([df["final_date"].dt.year]).count().plot(kind="bar", ax=ax)
        ax.set_xlabel("Years")
        ax.set_ylabel("Websites published with the term agil / agility")
        ax.set_title(org)   

plt.show()

In [ ]:
#Germany

In [37]:
# process statewise GERMAN VERSION 



def process_state_DE(state):

    agile_regex = re.compile(r'\bagility\b|\bagil\w{0,2}\b|\bagilität\b', re.IGNORECASE | re.UNICODE)

    agile_method_regex = re.compile(r'\bagil\w{0,2}\b((?![.]|agil\w{0,2}).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)

    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE)

    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/Germany/agile_sites_output_Germany_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])
    array = {}
    for line in csv_main: 
        array[line["id"]] = line
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/TextFiles/Germany/{}".format(state)
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)



######################## STILL TO BE ADAPTED FROM THE UK ######################





    for id, line in array.items():
        print("processing {}".format(id))
        id = line["id"]
        domain = line["domain"]
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/Germany/{}/{}/{}.html'.format(state, domain, id)
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            soup = BeautifulSoup(open(path, encoding='windows-1252'), "html.parser")    
                
        #deleting all the Script and Style Elements
        for script in soup(["script", "style", "a"]):
            script.decompose()    # rip it out
                
         # get text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
        #old 'simple' cleaning         
        #doctext = soup.get_text().replace("\n", " ")
        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        
        #Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        
        #Getting date5

        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date6

        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier

        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")

        #Getting date with the htmldate package
        htmldate = find_date(tree)


        # assinging the already crawled dates
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 


        #finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext)

        #removing white space, dot etc. from the end of the term
        
        signs=[' ', '.',':',';']
        
       # print(agile_term_match)
        agile_term=[]
        
        if agile_term_match != []:
            for item in agile_term_match:
                if item[-1] in signs:
                   # print("item", item)
                    newitem = item[:-1]
                   # print("newitem", newitem)
                else:
                    newitem = item
                newitem = newitem.lower()
                agile_term.append(newitem)

        #print("newitem", newitem)
        #print("agile_term",agile_term)

        #finding the matches for agile...methods
        agile_method = []
        agile_method = agile_method_regex.findall(doctext.lower())


        # finding the context for the agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all the variables to the items

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)
        
        if len(agile_method) == 0:
            line["agile_method"] = "" 
        else:
            line["agile_method"] = ",".join(agile_method)

        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "Germany"
        line["level"] = state
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
    
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}.csv".format(state), "w")
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'final_date', 'htmldate', 'heading', 'agile_term', 'agile_method', 'agile_context_pre', 'agile_context_post', 'text_file_loc'])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()


In [8]:
#Running processing for Germany
process_state_DE("Federal")

processing d64b7c1c-cddf-3ef7-ab10-f6c32100fc36
processing 001213a3-6bb1-374b-83ff-6d166d567933
processing c41d71f9-6ced-3c39-a661-dd85dff2175c
processing a36f0ef0-ec1b-33b6-aad4-45cda5defa48
processing 4d89205c-ee61-3d36-bdcc-d9ec9b22a082
processing 19dd6541-fe8a-3edb-835b-ab6d94423537
processing 447443cc-ba1b-3d2b-b0b7-5390f81e6569
processing af9dd497-d2a2-35ca-b48f-33c26d4476de
processing 9e8cc26a-36ba-3046-bf44-5c1484f2f0b3
processing 42f53033-7a47-39bf-ade6-3e3623083fb2
processing 1c69cbda-86f6-348a-ba31-a53f12a9197e
processing 83d20ea8-c770-37d6-b880-c7f329c5ced5
processing 9f67207b-61f1-3521-88c4-b533f74a244b
processing b0f2757b-c084-34a0-a104-e1bcbae826c8
processing 68d31c56-39f1-3753-b29d-ac5842eac9d3
processing 9e925771-d3a3-3b40-add7-fde6e74d97f2
processing f470ba9c-6aa9-39a0-86e5-d560d928cd16
processing 9b2b3bc8-d57f-345b-a4cc-66acb4144e91
processing 2233b421-720a-391b-b517-ae3a5233afe4
processing 242e8fc5-7383-3e83-bd2c-a00b7ee9b877
processing 635cbcf2-0fe2-39cd-91b5-d1ad7

In [3]:
##### Cleaning State Germany
def cleaning_state_DE(state):
    inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}.csv".format(state)
    outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}_v1.csv".format(state)

    with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
        df = pd.read_csv(inFile)
        df.sort_values(by=['final_date'])
        df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df['heading'] = df['heading'].astype(str)
        #deleting false positives
        df = df[~df.agile_context_post.str.startswith("(")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith(")")] # dropdown menu entry

        df.to_csv(outFile)

In [41]:
#Cleaning State Germany
cleaning_state_DE('insert_state_name')

NameError: name 'cleaning_state_DE' is not defined

In [ ]:
################### END OF PREPROCESSING ###################

In [ ]:
####### Combining all the Data Frames into one with pandas ########






In [28]:
#tester for getting dates with Xpath

htmlparser = etree.HTMLParser()    
try:
    tree = etree.parse(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/ALL_CRAWLS/Germany/Federal/www.westmidlands-pcc.gov.uk/765598f5-dc1c-3af2-94a6-96bd59ad0c44.html", "r"), htmlparser)
except UnicodeDecodeError:
    tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
date8 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")
print(date8)



2019-05-29T12:44:02+00:00


In [20]:
#testing regex expressions
agile_regex = re.compile(r'\bagile\b(?:(?![.]|agile).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)
text = 'agile teams            To help people in policy or related areas understand what digital means in government, exploring the methodology'
print(agile_regex.findall(text))

[' ']


In [4]:
#testing deletion of elements from HTML

markup = '<li class="gem-c-related-navigation__link"><a class="gem-c-related-navigation__section-link gem-c-related-navigation__section-link--sidebar  gem-c-related-navigation__section-link--other" data-track-category="relatedLinkClicked" data-track-action="1.4 Related content" data-track-label="/government/publications/assurance-for-agile-delivery-of-digital-services" href="/government/publications/assurance-for-agile-delivery-of-digital-services">Assurance for agile delivery of digital services</a></li>'
soup = BeautifulSoup(markup)
a_tag = soup.a

soup.i.decompose()

a_tag
# <a href="http://example.com/">I linked to</a>

AttributeError: 'NoneType' object has no attribute 'items'